### filtering commetns scraped from api 

In [25]:
import re
from glob import glob 
import pickle
import pandas as pd 
import re

slang_terms = [
    "LOL", "LMAO", "OMG", "OMFG", "WTF", "SMH", "YOLO", "FOMO", 
    "GG", "SUS", "STFU", "OMGOSH", "JK", "BFFL", "OFC", "ILY", "ILYSM", "XOXO"
]

def contains_emoji_or_slang(text):
    # Regular expression to match emojis
    emoji_pattern = re.compile(
        "[\U0001F600-\U0001F64F"  # emoticons
        "\U0001F300-\U0001F5FF"  # symbols & pictographs
        "\U0001F680-\U0001F6FF"  # transport & map symbols
        "\U0001F700-\U0001F77F"  # alchemical symbols
        "\U0001F780-\U0001F7FF"  # geometric shapes extended
        "\U0001F800-\U0001F8FF"  # supplemental arrows-C
        "\U0001F900-\U0001F9FF"  # supplemental symbols and pictographs
        "\U0001FA00-\U0001FA6F"  # chess symbols
        "\U0001FA70-\U0001FAFF"  # symbols and pictographs extended-A
        "\U00002702-\U000027B0"  # dingbats
        "\U000024C2-\U0001F251"  # enclosed characters
        "]+",
        flags=re.UNICODE,
    )
    
    # Check for emojis
    contains_emoji = bool(emoji_pattern.search(text))
    
    # Refined slang check: Slang terms that are used in context
    contains_slang = False
    for slang in slang_terms:
        # Check if slang is used in the text but not as part of other words (use word boundaries)
        if re.search(r'\b' + re.escape(slang) + r'\b', text, re.IGNORECASE):
            contains_slang = True
            break  # No need to continue once we find a match
    
    return contains_emoji or contains_slang



In [38]:
comments_path='comments_posts/comments_v2.csv'
commnets_df=pd.read_csv(comments_path)
commnets_df

,url,author,date,timestamp,score,upvotes,downvotes,golds,comment,comment_id,subreddit
0,https://oauth.reddit.com/r/artificial/comments...,johnfromberkeley,2024-11-26,2024-11-26 12:43:33,4,4,0,0,Fugatto Schmugatto. [Howard Dean was doing thi...,lz2ebeg,artificial
1,https://oauth.reddit.com/r/artificial/comments...,Captain_Cowboy,2024-11-26,2024-11-26 14:59:28,2,2,0,0,&gt; Factory machinery does not scream in ago...,lz2zm2h,artificial
2,https://oauth.reddit.com/r/artificial/comments...,DrummerHead,2024-11-26,2024-11-26 17:51:36,1,1,0,0,So I can finally turn my mouth sounds into act...,lz3wrzx,artificial
3,https://oauth.reddit.com/r/artificial/comments...,teSiatSa,2024-11-26,2024-11-26 12:33:14,15,15,0,0,And... What happened then? Was the AI used for...,lz2cyyp,artificial
4,https://oauth.reddit.com/r/artificial/comments...,dannown,2024-11-26,2024-11-26 12:33:04,10,10,0,0,"Um, this is 0% surprising.",lz2cy4y,artificial
...,...,...,...,...,...,...,...,...,...,...,...
8250,https://oauth.reddit.com/r/worldnews/comments/...,Pim_Hungers,2024-11-25,2024-11-25 20:17:07,9,9,0,0,So he wants to buy tens of billions of new gea...,lyyliop,worldnews
8251,https://oauth.reddit.com/r/worldnews/comments/...,FingalForever,2024-11-25,2024-11-25 20:06:37,-19,-19,0,0,Canada needs to speed up dramatically spending...,lyyjhg8,worldnews
8252,https://oauth.reddit.com/r/worldnews/comments/...,HighDeltaVee,2024-11-25,2024-11-25 20:57:07,10,10,0,0,"Russia : ""Our good friend in Iraq wil... wait,...",lyytdih,worldnews
8253,https://oauth.reddit.com/r/worldnews/comments/...,SteakForGoodDogs,2024-11-25,2024-11-25 23:25:08,13,13,0,0,"Okay, let 'em produce as much as they want.\n\...",lyzl1gn,worldnews


In [39]:
def replace_url(text): 
    text=re.sub('https://oauth','https://www',text)
    return text
commnets_df['url']=commnets_df['url'].apply(lambda x : replace_url(x))

In [40]:
commnets_df.to_csv(comments_path,index=False, encoding='utf-8-sig')

In [29]:
commnets_df.isnull().sum()

url           0
author        0
date          0
timestamp     0
score         0
upvotes       0
downvotes     0
golds         0
comment       0
comment_id    0
subreddit     0
dtype: int64

In [30]:
commnets_df.duplicated().sum()

0

In [31]:
commnets_df['has_emoji']=commnets_df['comment'].apply(lambda x : contains_emoji_or_slang(x) if pd.notna(x) else False)

In [32]:
commnets_df['has_emoji'].value_counts()

has_emoji
False    7830
True      425
Name: count, dtype: int64

In [36]:
commnets_df[commnets_df['has_emoji']==True].drop(columns=['has_emoji']).to_csv('comments_posts/final_comments.csv',index=False, encoding='utf-8-sig')

In [37]:
commnets_df[commnets_df['has_emoji']==True].drop(columns=['has_emoji']).to_excel('comments_posts/final_comments.xlsx',index=False)